# GTEx pipeline execution interface

## Preprocessing
See [this page](https://gaow.github.io/mvarbvs/doc/writeup/GTEx7_Analysis_Plan.html#Preprocessing) and [this meeting note](https://gaow.github.io/mvarbvs/doc/writeup/Meetings.html#Project-meeting-20170518) for details. 

In [5]:
%sossave prep.sos -f -x
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# Usage:
# ./prep.sos download
# ./prep.sos 

%include ResourceManagement as RM
%include Misc as MC
%include DataWrestling as DW

[global]

#
# Auxiliary steps
#

[download]
# Resource preparation
sos_run('RM.plink', workdir = CONFIG['wd'])
sos_run('RM.minimac3', workdir = CONFIG['wd'])
sos_run('RM.vcftools', workdir = CONFIG['wd'])
sos_run('RM.peer', workdir = CONFIG['wd'])
sos_run('RM.king', workdir = CONFIG['wd'])


#
# Workhorse
#

[data_summary]
input: CONFIG['genotype']
sos_run("MC.genotype_stats", workdir = CONFIG['wd'])

[genotype_preprocessing]
input: CONFIG['genotype']
sos_run("DW.vcf_by_chrom", workdir = CONFIG['wd'])

[rna_preprocessing]
input: CONFIG['rna_rpkm'], CONFIG['rna_cnts'], CONFIG['genotype'], CONFIG['sample_attr'] 
sos_run("MC.rnaseq", workdir =  CONFIG['wd'])

[pca_plot_broad]
input: '/tmp/GTExPCA.ped'
sos_run("MC.global_ancestry:2", workdir = '/tmp')

[genotype_pca_broad]
parameter: project_name = "GTEx7.dbGaP"
input: CONFIG['genotype']
sos_run("DW.vcf_by_chrom+DW.broad_to_plink+MC.LD_pruning+MC.global_ancestry", 
        workdir = CONFIG['wd'],
        project_name = project_name)

[genotype_pca_umich]
parameter: project_name = "GTEx7.Imputed"
input: CONFIG['imputed_genotype']
sos_run("DW.umich_to_plink+MC.LD_pruning+MC.global_ancestry", 
        workdir = CONFIG['wd'],
        project_name = project_name)

[genotype_pca_umich_filtered]
# Filtered imputation data removing imputed sites
input: "{}/GTEx7.Imputed.genotyped.filtered.bed".format(CONFIG['wd'])
sos_run("MC.LD_pruning+MC.global_ancestry", 
        workdir = CONFIG['wd'])

[gene_annotation: provides = "${CONFIG['rna_cnts']!n}.annotation"]
input: "${CONFIG['rna_cnts']}"
output: "${CONFIG['rna_cnts']!n}.annotation"
sos_run("MC.ensembl_annotation", workdir = CONFIG['wd'])

[genotype_formatting]
parameter: original_variants = "{}/GTEx7.dbGaP.bed".format(CONFIG['wd'])
parameter: gene_annotation = "${CONFIG['rna_cnts']!n}.annotation"
depends: original_variants
input: "{}/GTEx7.Imputed.bed".format(CONFIG['wd'])
sos_run("DW.variants_filter+DW.plink_to_hdf5_batch", 
        workdir = CONFIG['wd'], 
        include = original_variants,
        ann = gene_annotation)

Workflow saved to prep.sos


### Prepare computational resource

In [ ]:
!./prep.sos download -c conf/20170507.conf -b ~/Documents/GTEx/bin -J 8 -j 1

### Data summary

In [ ]:
!./prep.sos data_summary -c conf/20170507.conf -b ~/Documents/GTEx/bin -J 8 -j 1

### Genotype imputation

In [ ]:
!./prep.sos genotype_preprocessing -c conf/20170507.conf -b ~/Documents/GTEx/bin -J 8 -j 1

The imputation step was done with [Michigan Imputation Server](https://imputationserver.sph.umich.edu) because it uses Haplotype Reference Consortium (32,914 samples) reference panel which is not publicly available otherwise. So the genotype input would be genotype after imputation. [Here is how to prepare data](https://imputationserver.sph.umich.edu/start.html#!pages/help) for this service. The prepared files are uploaded to [Michigan imputation server](https://imputationserver.sph.umich.edu). [Here is configuration](https://gaow.github.io/mvarbvs/img/UMichImputation.png) of imputation job on UMich server, and [here](https://gaow.github.io/mvarbvs/img/UMichImputationResult.pdf) is summary of the outcome.

### RNA-seq preprocessing

In [ ]:
!./prep.sos rna_preprocessing -c conf/20170507.conf -b ~/Documents/GTEx/bin -J 8 -j 1

### Genotype file converstion and global ancestry analysis
`genotype_pca_umich_filtered` has to be executed after both `genotype_pca_umich` and `genotype_pca_broad` are executed.

In [ ]:
!./prep.sos genotype_pca_umich -c conf/20170507.conf -b ~/Documents/GTEx/bin/ -J 4
!./prep.sos genotype_pca_broad -c conf/20170507.conf -b ~/Documents/GTEx/bin/ -J 4
!./prep.sos pca_plot_broad
!./prep.sos genotype_pca_umich_filtered -c conf/20170507.conf -b ~/Documents/GTEx/bin/ -J 6

### Genotype QC, filtering / formatting

In [ ]:
!./prep.sos gene_annotation -c conf/20170507.conf -b ~/Documents/GTEx/bin -J 6 -j 1
!./prep.sos genotype_formatting -c conf/20170507.conf -b ~/Documents/GTEx/bin -J 6 -j 1